In [417]:
import numpy as np
from Bio import AlignIO
from Bio import SeqIO
from Bio.Seq import Seq
from bokeh.plotting import figure, show, output_notebook
# from bokeh.charts import Scatter, output_file, show
from bkcharts import Scatter, output_file, show
from bokeh.palettes import *
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, Band

import urllib
import time
import re


# re.compile("([/])").split(alignment.name)[0]

### Overview

- [Jackhmmer](#Jackhmmer)
- [Plot histogram](#Plot-Histogram)
- [Gap Filter prototype](#Gap-filter)
- [Coverage Calculation prototype](#Coverage)
    - [Run coverage prototype](#Run-coverage-prototype)
    - [Plot Coverage](#Plot-Coverage)
- [UniProt DB search for each MSA sequence](#Access-UniProt-DB-for-each-sequence-in-MSA)

## Jackhmmer

In [245]:
import subprocess
submit=("jackhmmer -N 1 -A %s --incE 1E-10 %s ../database/uniref90.fasta.gz"
        % (directory, directory,pdbID))

process = subprocess.Popen(submit.split(), stdout=subprocess.PIPE)
process.communicate()

NameError: name 'directory' is not defined

## Plot Histogram

In [93]:
def plotHist(array, num_bins=50, plot_title='Sequence length Coverage', 
             xlabel='Seq/Len', ylabel='Counts'):
    
    hist, edges = np.histogram(array, bins=num_bins)
    is_normed = np.sum(hist)
    print('Counts add to: %d' % is_normed)
    hover = HoverTool(tooltips=[('x-axis, Counts', '$x{0.00}, $y{0.00}')])
    TOOLS="crosshair,pan,wheel_zoom,reset,save,box_select"
    hq = figure(title=plot_title, 
               width=800, height=300, tools=[TOOLS, hover],
               background_fill_color='beige')

    hq.xaxis.axis_label = xlabel
    hq.yaxis.axis_label = ylabel

    hq.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
            fill_color="teal", line_color="#033649", alpha=0.5)

    hq.ygrid.band_fill_alpha = 0.1
    hq.ygrid.band_fill_color = "navy"
    hq.legend.click_policy="hide"
    output_notebook()
    show(hq)

[Back to top](#Overview)

## Gap filter

In [442]:
def gap_filter(msa_input, gap_threshold=0.25, output_gaps=False):
    print('================================================================')
    print('Gap filter for Multiple-Sequence Alignment based on percent gaps')
    print('Gap threshold is %2.d%%.\n' % (gap_threshold*100))
    length_sequence = 375
    alignment = AlignIO.read(msa_input, 'fasta')
    total_sequences= len(alignment)


    output_handle = open(msa_input.name + '_filtered_%dp' % (gap_threshold*100), 'w')
    
    print('Number of sequences: %d' % total_sequences)
    print('Actin sequence length: %d' % length_sequence)
    print('\n')
    
    filtered_sequences = 0
    percent_gaps = np.zeros(total_sequences)
    
    for seq_num, record in enumerate(alignment):
        # Counts number of gaps in current sequence
        # Note: removes insertions via ungap method 
        current_sequence = record.seq.ungap('.')
        temp_seq = Seq('')

        
        # Removes lowercase letters
        for residue in range(len(current_sequence)):
            if (current_sequence[residue].islower() == False):
                temp_seq = temp_seq + current_sequence[residue]
                
        current_sequence = temp_seq
        gap_count = current_sequence.count('-')

        # Calculates % gaps in current sequence
        percent_gaps[seq_num] = float(gap_count) / float(len(current_sequence))
        
        # Outputs current sequence that maintains minimum gap threshold
        if (percent_gaps[seq_num] <= gap_threshold):
            filtered_sequences += 1
            output_handle.write('>%s\n' % record.id + 
                                ''.join(current_sequence) + '\n')

    removed_sequences = total_sequences - filtered_sequences
    
    print('Number of sequences removed: %d(%s%%)' % (
        removed_sequences, removed_sequences*100/total_sequences))
    print('Number of sequences kept: %d\n' % filtered_sequences)
    print('Wrote new file: \'' + output_handle.name + '\'\n')    
    output_handle.close()
    msa_input.close()
    
    if (output_gaps == True):
        return percent_gaps

[Back to top](#Overview)

In [444]:
# folder = '/home/kmm5/actin/actin-sequences/pfam-MSA/pfam_31/'
# msa_input_1 = folder + 'PF00022_full_pfam31.afa'
# msa_input_5 = '/home/kmm5/actin/actin-sequences/jackhmmer/actin-actin_domain/actinx2_09-25-17.afa'
# msa_input_6 = '/home/kmm5/actin/actin-sequences/jackhmmer/actin_single_domain/actin_single.afa'
# # msa_input = [msa_input_1, msa_input_5, msa_input_6]

# # for i in range(len(msa_input)): 
# gap_filter(open(folder + 'PF00022_rp35.txt'))

## Coverage

[Back to top](#Overview)

In [262]:
def calculate_coverage(msa_input):
    
    length_sequence = 375
    alignment = AlignIO.read(msa_input, 'fasta')
    total_sequences= len(alignment)
    length_alignment = alignment.get_alignment_length()
    total_possible_coverage = float(total_sequences) / float(length_sequence)

    print('================================================================')
    print('Number of sequences: %d' % total_sequences)
    print('Alignment length %d' % length_alignment)
    print('Actin sequence length: %d' % length_sequence)
    print("Seq/Len: %d\n" % total_possible_coverage)

    filtered_sequences = 0
    site_coverage = np.zeros((total_sequences, length_alignment))
    percent_gaps = np.zeros(total_sequences)

    for i, record in enumerate(alignment):
        residue_count = 0
        gap_count = 0
        current_sequence = record.seq
        # Loop thru every residue in current sequence and calculate site coverage
        for residue in range(length_alignment):
            if (current_sequence[residue] == '-'):
                gap_count += 1
            else:
                residue_count += 1
                site_coverage[i,residue] = float(i) / float(length_sequence)

    msa_input.close()
    avg_site_coverage = np.zeros(length_alignment)
    for i in range(length_alignment):
        avg_site_coverage[i] = np.sum(site_coverage[:, i]) / float(total_sequences)
    print('Average sequence coverage: %.2f\n' % np.max(avg_site_coverage))
    
    return avg_site_coverage

### Run coverage prototype

[Back to top](#Overview)

In [241]:
folder = '/home/kmm5/actin/actin-sequences/pfam-MSA/pfam_31/'
msa_input_1 = folder + 'PF00022_full_pfam31.afa'
msa_input_2 = folder + 'PF00022_full_pfam31.afa_f10'
msa_input_3 = folder + 'PF00022_full_pfam31.afa_f20'
msa_input_4 = folder + 'PF00022_full_pfam31.afa_f50'
# msa_input_5 = '/home/kmm5/actin/actin-sequences/jackhmmer/actin-actin_domain/actinx2_09-25-17.afa'
# msa_input_6 = '/home/kmm5/actin/actin-sequences/jackhmmer/actin_single_domain/actin_single.afa'

msa_input = [msa_input_1, msa_input_2, msa_input_3, msa_input_4]
a_site_coverage = np.zeros((len(msa_input), 407))

for i in range(len(msa_input)):
    a_site_coverage[i] = calculate_coverage(open(msa_input[i]))



Number of sequences: 14516
Alignment length 407
Actin sequence length: 375
Seq/Len: 38

Average sequence coverage: 14.86

Number of sequences: 788
Alignment length 407
Actin sequence length: 375
Seq/Len: 2

Average sequence coverage: 1.05

Number of sequences: 1145
Alignment length 407
Actin sequence length: 375
Seq/Len: 3

Average sequence coverage: 1.53

Number of sequences: 7108
Alignment length 407
Actin sequence length: 375
Seq/Len: 18

Average sequence coverage: 9.48



In [ ]:
# a_site_coverage[1]

## Plot Coverage

[Back to top](#Overview)

In [ ]:
def plotSiteCoverage(array, x_range, total_coverage):
    """
    Plots the ratios given an array of
    len(ratio_array) and an x-axis range.
    """
    colorlist = Dark2_8
    hover = HoverTool(tooltips=[('Top Pairs, IMatch Ratio', '$x{0}, $y{0.00}')])
    TOOLS = "crosshair, pan, wheel_zoom, reset, save, box_select"
    p = figure(title='Single-Site Coverage', width=800, height=300, 
              tools=[TOOLS, hover], toolbar_location="above",
              x_axis_label='Amino Acid Site', x_range=[0,x_range],
              y_axis_label='Fully-Covered Seqs')
    
#     p.line(range(x_range), total_coverage, line_dash='dashed', line_width=1.2, color='firebrick')
    
    for i in range(len(array)):
        p.line(range(x_range), array[i], color=colorlist[i],
               legend=('msa_%d' % i), line_width=1)
        
    p.background_fill_color='beige'
    p.legend.location='top_left'
    p.xgrid.grid_line_color = 'navy'
#     p.xgrid.grid_line_dash = 'dashed'
    p.xgrid.grid_line_alpha = 0.1
    p.ygrid.band_fill_alpha = 0.1
    p.ygrid.band_fill_color = "navy"
    p.legend.orientation='vertical'
    p.legend.click_policy='hide'
    output_notebook()
    show(p)

In [242]:
plotSiteCoverage(a_site_coverage, 407, 38)

Loading BokehJS ...

In [33]:
plotSiteCoverage(a_site_coverage, length_sequence, coverage)

Loading BokehJS ...

In [39]:
plotHist(p_gaps, num_bins=50, plot_title='Fraction of Sequence Gaps', xlabel='Fraction Gaps')

Counts add to: 5460


Loading BokehJS ...

In [38]:
plotSiteCoverage(a_site_coverage, length_sequence, coverage)

Loading BokehJS ...

In [46]:
plotSiteCoverage(a_site_coverage, length_sequence, coverage)

Loading BokehJS ...

In [37]:
# alignment_stk = AlignIO.read(open(folder + 'PF00022_full_pfam31.stk'), format='stockholm')
# l_a = alignment_stk.get_alignment_length()
# print("Alignment length: %d" % l_a)
# r = alignment_stk[2]
# l_a - r.seq.count('-') / float(3445)

## Access UniProt DB for each sequence in MSA

[Back to top](#Overview)

In [446]:
# count = 0
# t0 = time.time()
# for record in alignment[:2]:
#     #print(record.id)
#     #handle = urllib.urlopen("http://www.uniprot.org/uniprot/"+ record.id +".xml")
# #     try:
#     record_id = re.compile("([/])").split(record.id)[0]
#     uni_record = SeqIO.read(urllib.urlopen("http://www.uniprot.org/uniprot/"+ record_id +".xml"), "uniprot-xml")
#     names.append(record.id)
#     descriptions.append(uni_record.description)
# #     except:
# #         pass
#     count += 1
# t1 = time.time()
# total = t1-t0
# print "Parsed %d sequences." % count
# print "Time elapsed: %f seconds.\n" % total

## Write names and descriptions to file

[Back to top](#Overview)

In [ ]:
# output = open("./actin-sequences/jackhmmer/actin-mreb/test.txt", "w")
# for i in xrange(count):
#     #if unknown description dont print (implement in the future)
#     output.write(names[i] + "\n" + 'Description: ' + descriptions[i] + "\n\n")
# output.close()
# print "Writing names and descriptions to file."

In [ ]:
# msa2 = "./actin-sequences/jackhmmer/actin-actin_domain/actin-actin_cyt.afa_filtered50"
# alignment2 = AlignIO.read(open(msa2), "fasta")
# for record2 in alignment[:5]:
#     print record2.id
    

# #N2 = len(alignment2)
# #for j in np.arange(N2):
#     #print j